In [159]:
import numpy as np
import tensorflow as tf
import math
from skimage.transform import resize
from PIL import Image
import h5py
from tensorflow.python.framework import ops
import scipy
import matplotlib.pyplot as plt
import matplotlib.image as img
from scipy import ndimage
import os
import cv2
import random
from sklearn.utils import shuffle
import pickle as pkl

In [28]:
np.random.seed(1)
x=np.random.rand(4)
np.random.seed(1)
y=np.random.rand(4)
print(x)
print(y)

[4.17022005e-01 7.20324493e-01 1.14374817e-04 3.02332573e-01]
[4.17022005e-01 7.20324493e-01 1.14374817e-04 3.02332573e-01]


In [168]:
def load_dataset(isSave):
    
    # isSave will be true when we want to save the data in pkl and will be false when 
    # we want to get the data from pkl
    if isSave:
        classes=os.listdir("leaf_dataset")

        n_class=len(classes)
        samples_per_class=16
        image_width=20 # width of image
        image_height=20 # height of image
        m=n_class*samples_per_class # Number of samples))

        train_ratio=0.7
        validation_ratio=0.2
        test_ratio=0.1

        ntraining_samples=int(samples_per_class*train_ratio)
        nvalidation_samples=int(samples_per_class*validation_ratio)
        ntest_samples=samples_per_class-ntraining_samples-nvalidation_samples

        print("m",m)
        print("classes",len(classes))
        print("ntraining_samples",ntraining_samples)
        print("nvalidation_samples",nvalidation_samples)
        print("ntest_samples",ntest_samples)

        x_train_set=np.zeros((ntraining_samples*n_class,image_width,image_height))
        x_validation_set=np.zeros((nvalidation_samples*n_class,image_width,image_height))
        x_test_set=np.zeros((ntest_samples*n_class,image_width,image_height))

        y_train_set=np.zeros((ntraining_samples*n_class,1),dtype=int)
        y_validation_set=np.zeros((nvalidation_samples*n_class,1),dtype=int)
        y_test_set=np.zeros((ntest_samples*n_class,1),dtype=int)

        train_count=0
        test_count=0
        validation_count=0

        for count,class_name in enumerate(classes):
            class_name_path="leaf_dataset/"+class_name+"/"

            images=os.listdir(class_name_path);
            random.shuffle(images)

            train_images=images[:ntraining_samples]
            validation_images=images[ntraining_samples:ntraining_samples+nvalidation_samples]
            test_images=images[ntraining_samples+nvalidation_samples:]

#             print("train_images",train_images)
#             print("validation_images",validation_images)
#             print("test_images",test_images)

            # Filling training set
            for image_name in train_images:
    #             print("image_name",image_name)
                image=img.imread(class_name_path+image_name)
    #             print(image.shape)
                image=cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
    #             print(image.shape)
                x_train_set[train_count]=image
                y_train_set[train_count]=count
                train_count+=1

            # Filling validation set
            for image_name in validation_images:
    #             print("image_name",image_name)
                image=img.imread(class_name_path+image_name)
    #             print(image.shape)
                image=cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
    #             print(image.shape)
                x_validation_set[validation_count]=image
                y_validation_set[validation_count]=count
                validation_count+=1

            # Filling test set
            for image_name in test_images:
    #             print("image_name",image_name)
                image=img.imread(class_name_path+image_name)
    #             print(image.shape)
                image=cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
    #             print(image.shape)
                x_test_set[test_count]=image
                y_test_set[test_count]=count
                test_count+=1

#         print(len(classes))
    #     np.set_printoptions(threshold=np.inf)
#         print("y_train_set",y_train_set.shape,y_train_set)
#         print("y_validation_set",y_validation_set.shape,y_validation_set)
#         print("y_test_set",y_test_set.shape,y_test_set)

        x_train_set,y_train_set = shuffle(x_train_set,y_train_set, random_state=0) # This shuffles two array altoghether 
        x_validation_set,y_validation_set = shuffle(x_validation_set,y_validation_set, random_state=0)
        x_test_set,y_test_set = shuffle(x_test_set,y_test_set, random_state=0)
        
        with open("xy_train_set.pkl",'wb') as f:
            pkl.dump((x_train_set,y_train_set),f)
            
        with open("xy_validation_set.pkl",'wb') as f:
            pkl.dump((x_validation_set,y_validation_set),f)
            
        with open("xy_test_set.pkl",'wb') as f:
            pkl.dump((x_test_set,y_test_set),f)
        print("Data saved successfully")
        return x_train_set,y_train_set,x_validation_set,y_validation_set,x_test_set,y_test_set
            
    else:       
        with open("xy_train_set.pkl",'rb') as f:
            x_train_set,y_train_set=pkl.load(f)
        
        with open("xy_validation_set.pkl",'rb') as f:
            x_validation_set,y_validation_set=pkl.load(f)
            
        with open("xy_test_set.pkl",'rb') as f:
            x_test_set,y_test_set=pkl.load(f)
        
        print("Data loaded successfully")
        return x_train_set,y_train_set,x_validation_set,y_validation_set,x_test_set,y_test_set


In [167]:
x_train_set,y_train_set,x_validation_set,y_validation_set,x_test_set,y_test_set=load_dataset(isSave=False)
print(x_train_set,y_train_set,x_validation_set,y_validation_set,x_test_set,y_test_set)

[[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.   0. 255. 255. 255. 255. 255.   0.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255.
   255.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   3. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255. 255.   0.   0.   0.]
  [  0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255. 255.   0.   0.   0.]
  [  0.   0.   0. 215. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255. 255. 255.   0.   0.]
  [  0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255. 255. 255. 255.   0.]
  [  0.   0. 255. 255. 255. 255. 255.

 [62]] [[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   1. 255.   0.   0.   0.   0.   0.   0.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0. 255. 255. 255. 255. 255. 254.   0.   0.   0.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255.   0.   0.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255.   3.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255.   0.   0.   0.   0.   0.]
  [  0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255.   0.   0.   0.   0.]
  [  0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255. 255.   0.   0.   0.]
  [  0.   0. 129. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255. 255. 255.   1.   0.]
  [  0.   1. 255. 255. 255. 25

 [57]] [[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 255.   0.   0.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 255. 255.   0.   0.
     0.   0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255.
   255. 152.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255.   0.   0.   0.   0.]
  [  0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255.   0.   0.   0.   0.]
  [  0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255. 255.   1.   0.   0.]
  [  0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255. 255. 255.   0.   0.]
  [  0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
   255. 255. 255. 255.   0.   0.]
  [  0. 255. 255. 255. 255. 25

In [108]:
print("l"+"b")

lb


In [153]:
l=[[1,2],[3,4],[2,3],[4]]
b=[1.4,2.4,3.4,4.4]
from sklearn.utils import shuffle
l, b = shuffle(l, b, random_state=0)
print(l)
print(b)

[[2, 3], [4], [3, 4], [1, 2]]
[3.4, 4.4, 2.4, 1.4]


In [117]:
for i in enumerate(l):
    

1
1
1
1


In [161]:
with open("test.pkl",'wb') as f:
    pkl.dump((l,b),f)

In [162]:
with open("test.pkl",'rb') as f:
    l,b=pkl.load(f)
    print(l)
    print(b)

[[2, 3], [4], [3, 4], [1, 2]]
[3.4, 4.4, 2.4, 1.4]
